# Network Intrusion Detection with Deep Learning

In [1]:
import os


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

C:\ProgramData\Anaconda3\envs\talos\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html


## The Data

In [3]:
import numpy as np
def load_matrices(file_name):
    with open(file_name, 'rb') as f:
        A = np.load(f)
        B = np.load(f)
        C = np.load(f)
        D = np.load(f)
        E = np.load(f)
        F = np.load(f)
    return (A,B,C,D,E,F)

In [4]:
my_file =  '../CIDS-2018/10data2.npy'
train,test,y_train,y_test,ylabel_train, ylabel_test = load_matrices(my_file)

## The Model

In [5]:
from keras import backend as K
#from keras.layers import Dense, Activation, Merge, Reshape, Dropout
from keras.layers import Activation, Reshape, Input, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers import InputLayer, Input
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
from keras.models import load_model
from keras.models import Model, Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import PReLU
from keras.activations import elu
from keras.losses import binary_crossentropy

Using TensorFlow backend.


In [6]:
import talos as ta
import wrangle as wr
from talos.metrics.keras_metrics import fmeasure_acc
from talos import live

In [9]:
def selu(x):
    """Scaled Exponential Linear Unit. (Klambauer et al., 2017)
    # Arguments
        x: A tensor or variable to compute the activation function for.
    # References
        - [Self-Normalizing Neural Networks](https://arxiv.org/abs/1706.02515)
    """
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale * elu(x, alpha)

#model.add(Dense(32, input_shape=(input_length - 1,)), activation=selu)

In [10]:
#actvation leakyrelu
LR= LeakyReLU(0.01)
LR.__name__ = 'lrelu'


In [12]:
from keras.callbacks import ModelCheckpoint, CSVLogger


In [13]:
def log_file_name1(first_neuron,second_neuron,third_neuron, batch_size,kernel_initial,lr,activation):
    
    if activation==LR:
       activation= 'lrelu'
    
    if activation==PReLU:
       activation= 'prelu'
    
    if activation==relu:
       activation= 'relu'
    
    if activation==elu:
       activation= 'elu'
    
    if activation==selu:
       activation= 'selu'

    # The dir-name for the TensorBoard log-dir.
    s = "./logtalos/log5a/neuron1_{0}-neuron2_{1}-neuron3_{2}-batch{3}-kernelinit_{4}-lr{5}-activ_{6}.hdf"
    log_file = s.format(first_neuron,second_neuron,third_neuron,batch_size,kernel_initial,lr,activation)
    return log_file

def log_file_name2(first_neuron,second_neuron,third_neuron, batch_size,kernel_initial,lr,activation):
    
    if activation==LR:
       activation= 'lrelu'
    
    if activation==PReLU:
       activation= 'prelu'
    
    if activation==relu:
       activation= 'relu'
    
    if activation==elu:
       activation= 'elu'
    
    if activation==selu:
       activation= 'selu'
    
    # The dir-name for the TensorBoard log-dir.
    s = "./logtalos/log5a/neuron1_{0}-neuron2_{1}-neuron3_{2}-batch{3}-kernelinit_{4}-lr{5}-activ_{6}.csv"
    log_file = s.format(first_neuron,second_neuron,third_neuron, batch_size,kernel_initial,lr,activation)
    return log_file


In [17]:
from keras.callbacks import ModelCheckpoint
filepath="./logtalos/5ates_a.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [18]:
from keras.models import Model, Sequential

In [19]:
# first we have to make sure to input data and params into the function
# add input parameters to the function
def dnn(x_train, ylabel_train, x_val, y_val, params):

    input_dim = x_train.shape[1]

    input_img = Input(shape=(input_dim,))
    
    first_neuron=params['first_neuron']
    second_neuron=params['second_neuron']
    third_neuron=params['third_neuron']
    batch_size=params['batch_size']
    kernel_initial=params['kernel_initializer']
    lr=params['lr']
    activation=params['activation']
    
    print('activation=',activation)
    
    log_file1 = log_file_name1(first_neuron,second_neuron,third_neuron, batch_size,kernel_initial,lr,activation)
    checkpoint2 = ModelCheckpoint(log_file1, monitor='acc', save_best_only=True, mode='max')

    
    log_file2 = log_file_name2(first_neuron,second_neuron,third_neuron,batch_size,kernel_initial,lr,activation)

    csv_logger2 = CSVLogger(log_file2,separator=',', append=False)
    callbacks_list2 = [checkpoint2,csv_logger2]
    
    ###AE
    
    # replace the hyperparameter inputs with references to params dictionary 
    autoencoder = Sequential()
    #encoder
    
    if activation==PReLU: 
        autoencoder.add(Dense(first_neuron, input_shape=(input_dim,), kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(second_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(third_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
                   
        #decoder
        #model.add(Dense(8, activation=params['activation']))
        autoencoder.add(Dense(second_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(first_neuron, kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        autoencoder.add(Dense(input_dim, activation='sigmoid', kernel_initializer=kernel_initial))
        
    else:
        autoencoder.add(Dense(first_neuron, input_shape=(input_dim,), activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(second_neuron, activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(third_neuron, activation=activation, kernel_initializer=kernel_initial))
        #decoder
        #model.add(Dense(8, activation=params['activation']))
        autoencoder.add(Dense(second_neuron, activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(first_neuron, activation=activation, kernel_initializer=kernel_initial))
        autoencoder.add(Dense(input_dim, activation='sigmoid', kernel_initializer=kernel_initial))
    
    #model = multi_gpu_model(model, gpus=1)
    autoencoder.summary()
    print(input_dim)
    
    input_img = Input(shape=(input_dim,))
    encoder_layer1 = autoencoder.layers[0]
    encoder_layer2 = autoencoder.layers[1]
    encoder_layer3 = autoencoder.layers[2]
    encoder = Model(input_img, encoder_layer3(encoder_layer2(encoder_layer1(input_img))))
    
    #encoder_layer1 = autoencoder.layers[0]
    #encoder_layer2 = autoencoder.layers[1]
    #encoder_layer3 = autoencoder.layers[2]
    #encoder = Model(input_img, encoder_layer3(encoder_layer2(encoder_layer1(input_img))))
    encoder.summary()

    
    autoencoder.compile(loss='mse', optimizer='adam', metrics=['acc'])
    #model.compile(loss=params['losses'],
    #              optimizer=params['optimizer'](),
    #              metrics=['acc', fmeasure_acc])
    
   

    
    history=autoencoder.fit(x,x,
                             epochs=params['epochs'],
                             batch_size=batch_size,
                              #shuffle=True,
                              validation_split=0.2,
                              callbacks=callbacks_list,                    
                             #callbacks=[early_stopping])
                              #validation_data=(test,test),
                              verbose=1)
  

    num_classes=15
    out2 = Dense(num_classes, activation='softmax',kernel_initializer=kernel_initial)(encoder.output)
    newmodel = Model(encoder.input,out2)
    
   
    newmodel.compile(loss='categorical_crossentropy', 
                     #optimizer='adam', 
                     optimizer= Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08,decay=0.0, amsgrad=False),
                     metrics=['accuracy']) 
    newmodel.summary()


    history2a=newmodel.fit(x, y,
                           epochs=params['epochs2'],
                           batch_size=batch_size,
                           validation_split=0.2,
                           callbacks=callbacks_list2,
                           #callbacks=[early_stopping],
                           #validation_data=(test,y_test),
                           verbose=1)

    
    del encoder
    del autoencoder
    #del dnn_model
    
    tf.reset_default_graph()
    
    return history2a, newmodel #, encoder
    

In [21]:
# then we can go ahead and set the parameter space
p = {'first_neuron':[70, 60, 50],
     'second_neuron':[40,35,30],
     'third_neuron':[25,20],
     #'first_neuron':[100,80],
     'batch_size': [32,64,256],
     #'batch_size': [32,64],
     'epochs': [20],
     'epochs2': [30],
     'dropout': [0],
     'kernel_initializer': ['lecun_uniform','lecun_normal','he_normal','he_uniform','normal','glorot_uniform','glorot_normal'],
     'optimizer': [Adam],
     'lr':[0.1,0.01, 0.001,0.0001],#0.00001],
     'activation':[relu, elu,selu, LR, PReLU]}
     #'activation':[relu]}#, elu,selu]}

In [22]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, clear_session

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True#True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [23]:
x=train
y=ylabel_train

In [24]:
np.shape(x)

(1619315, 80)

In [25]:
np.shape(y)

(1619315, 15)

In [26]:


t = ta.Scan(x, y, p, dnn, experiment_no='51',grid_downsample=0.05,)


  0%|          | 0/75 [00:00<?, ?it/s]

activation= <function relu at 0x00000212FE883488>
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                4050      
_________________________________________________________________
dense_2 (Dense)              (None, 40)                2040      
_________________________________________________________________
dense_3 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_4 (Dense)              (None, 40)                840       
_________________________________________________________________
dense_5 (Dense)              (None, 50)                2050      
_________________________________________________________________
dense_6 (Dense)              (None, 80)                4080      
Total params: 13,880

  1%|▏         | 1/75 [17:13<21:14:13, 1033.16s/it]

activation= <function relu at 0x00000212FE883488>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                4050      
_________________________________________________________________
dense_2 (Dense)              (None, 30)                1530      
_________________________________________________________________
dense_3 (Dense)              (None, 25)                775       
_________________________________________________________________
dense_4 (Dense)              (None, 30)                780       
_________________________________________________________________
dense_5 (Dense)              (None, 50)                1550      
_________________________________________________________________
dense_6 (Dense)              (None, 80)                4080      
Total params: 12,765
Trainable params: 12,765
Non-trainable params: 0
______________________

  3%|▎         | 2/75 [34:14<20:52:43, 1029.64s/it]

...

activation= <function relu at 0x00000212FE883488>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 60)                4860      
_________________________________________________________________
dense_2 (Dense)              (None, 40)                2440      
_________________________________________________________________
dense_3 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_4 (Dense)              (None, 40)                840       
_________________________________________________________________
dense_5 (Dense)              (None, 60)                2460      
_________________________________________________________________
dense_6 (Dense)              (None, 80)                4880      
Total params: 16,300
Trainable params: 16,300
Non-trainable params: 0
______________________

 99%|█████████▊| 74/75 [83:29:27<46:31, 2791.40s/it]  

activation= <function relu at 0x00000212FE883488>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                4050      
_________________________________________________________________
dense_2 (Dense)              (None, 35)                1785      
_________________________________________________________________
dense_3 (Dense)              (None, 20)                720       
_________________________________________________________________
dense_4 (Dense)              (None, 35)                735       
_________________________________________________________________
dense_5 (Dense)              (None, 50)                1800      
_________________________________________________________________
dense_6 (Dense)              (None, 80)                4080      
Total params: 13,170
Trainable params: 13,170
Non-trainable params: 0
______________________

100%|██████████| 75/75 [84:12:27<00:00, 4041.96s/it]

Scan Finished!


In [ ]:
x.shape

In [ ]:
t.data

In [ ]:
r = ta.Reporting(t)

In [ ]:
r.best_params()